In [3]:
from google.colab import drive
drive.mount("/content/drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [4]:
!ls

drive  sample_data


In [0]:
import os 

In [0]:
os.chdir("/content/drive/My Drive/Sentiment Analysis Code")

In [7]:
!ls

'Bag_Of_Words .ipynb'   positive.txt	     TextBlob_VADER.ipynb
 labeledTrainData.tsv   SentiWordnet.ipynb   VADER_Sentiment.ipynb
 negative.txt	        Text_Blob.ipynb


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv("labeledTrainData.tsv",header=0, delimiter="\t", quoting=3)

In [10]:
data.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( data['review'], data['sentiment'], test_size=0.33, random_state=42)


In [0]:

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [0]:
import string
from nltk.corpus import stopwords

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [17]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [18]:
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


True

In [0]:
def clean_text(text):
    text = text.replace("<br />", " ")
    #text = text.decode(encoding = 'UTF-8',errors = 'strict')
 
    return text


In [0]:
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [21]:
wn.ADJ

'a'

In [0]:
lemmatizer = WordNetLemmatizer()

"""
Method to Return a sentiment polarity: 0 = negative, 1 = positive
"""
def swn_polarity(text):
    sentiment = 0.0
    tokens_count = 0
    text = clean_text(text)
    #print(type(text))
    raw_sentences = sent_tokenize(text)
    #print(raw_sentences)
    for raw_sentence in raw_sentences:
      tagged_sentence = pos_tag(word_tokenize(raw_sentence))
      #print(tagged_sentence)
      for word, tag in tagged_sentence:
        #print(word,tag)
        wn_tag = penn_to_wn(tag)
        #print(wn_tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
          continue
        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        #print(lemma)
        if not lemma:
          continue
        synsets = wn.synsets(lemma, pos=wn_tag)
        #print(synsets)
        if not synsets:
          continue
        synset = synsets[0]
        #print(synset)
        swn_synset = swn.senti_synset(synset.name())
        #print(swn_synset)
        sentiment += swn_synset.pos_score() - swn_synset.neg_score()
        #print(sentiment)
        tokens_count += 1 
    if not tokens_count:
      return 0
    if sentiment >= 0:
      return 1
    return 0


In [23]:
swn_polarity(X_train[0])



0

In [0]:
# Predictions
pred_y = [swn_polarity(text) for text in data['review']]

In [25]:
from sklearn.metrics import accuracy_score

print(accuracy_score(data['sentiment'], pred_y)) 

0.6638


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC

In [0]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="word",
                                   tokenizer=word_tokenize,        
                                   #tokenizer=lambda text: mark_negation(word_tokenize(text)), 
                                   preprocessor=lambda text: text.replace("<br />", " "),
                                   ) ),
    ('classifier', SVC())  
])

In [28]:
pipeline.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1),
        preprocessor=<function <lambda> at ...f', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [29]:
pipeline.score(X_test, y_test)

0.6912727272727273

In [41]:
#print(pipeline.named_steps['vectorizer'].get_feature_names())

['\x08\x08\x08\x08A', '!', '#', '$', '%', '&', "'", "''", "''After", "''Cannibal", "''Empire", "''Family", "''I", "''Ned", "''Peeping", "''Return", "''Saint", "''Scarface", "''Talent", "''The", "''This", "''We", "''Zero", "''bad", "''clients", "''heart", "''human", "''humans", "''little", "''negative", "''on", "''oversexed", "''professionals", "''troubled", "''villain", "''voyeur", "''while", "'*name", "'0/10", "'00s", "'01", "'02", "'03", "'04", "'05", "'06", "'08", "'1", "'10", "'12", "'1408", "'1902", "'1909", "'1940", "'2", "'2001", "'24", "'28", "'30", "'30s", "'30s-'40s", "'30s-early", "'34", "'38", "'39", "'3rd", "'4", "'4-dimensional", "'40", "'40's/'50", "'40s", "'41", "'44", "'45", "'46", "'47", "'48", "'5", "'50", "'50s", "'50s-'60s", "'51", "'54", "'54-'55", "'56", "'60", "'60ies", "'60s", "'60s.\\", "'60s/'70s", "'60s/early", "'60s\\", "'61", "'62", "'63", "'65", "'66", "'68", "'69", "'70", "'70ies", "'70s", "'70s.\\", "'70s/'80s", "'70s\\", "'71", "'71\\", "'73", "'73\\",